<a href="https://colab.research.google.com/github/officialadhyayansaha-eng/Architecting-LLMs-WiDS-ProjectWork/blob/main/PokemonNamesandTemperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Implemented Trigram Makemore on Pokemon (GEN-1 to GEN-3)
#Added Temperature Factor (Optimal Names at temp=)

In [ ]:
pokemon_list = [
    "bulbasaur", "ivysaur", "venusaur", "charmander", "charmeleon", "charizard",
    "squirtle", "wartortle", "blastoise", "caterpie", "metapod", "butterfree",
    "weedle", "kakuna", "beedrill", "pidgey", "pidgeotto", "pidgeot", "rattata",
    "raticate", "spearow", "fearow", "ekans", "arbok", "pikachu", "raichu",
    "sandshrew", "sandslash", "nidoran", "nidorina", "nidoqueen", "nidorino",
    "nidoking", "clefairy", "clefable", "vulpix", "ninetales", "jigglypuff",
    "wigglytuff", "zubat", "golbat", "oddish", "gloom", "vileplume", "paras",
    "parasect", "venonat", "venomoth", "diglett", "dugtrio", "meowth", "persian",
    "psyduck", "golduck", "mankey", "primeape", "growlithe", "arcanine", "poliwag",
    "poliwhirl", "poliwrath", "abra", "kadabra", "alakazam", "machop", "machoke",
    "machamp", "bellsprout", "weepinbell", "victreebel", "tentacool", "tentacruel",
    "geodude", "graveler", "golem", "ponyta", "rapidash", "slowpoke", "slowbro",
    "magnemite", "magneton", "farfetchd", "doduo", "dodrio", "seel", "dewgong",
    "grimer", "muk", "shellder", "cloyster", "gastly", "haunter", "gengar",
    "onix", "drowzee", "hypno", "krabby", "kingler", "voltorb", "electrode",
    "exeggcute", "exeggutor", "cubone", "marowak", "hitmonlee", "hitmonchan",
    "lickitung", "koffing", "weezing", "rhyhorn", "rhydon", "chansey", "tangela",
    "kangaskhan", "horsea", "seadra", "goldeen", "seaking", "staryu", "starmie",
    "mrmime", "scyther", "jynx", "electabuzz", "magmar", "pinsir", "tauros",
    "magikarp", "gyarados", "lapras", "ditto", "eevee", "vaporeon", "jolteon",
    "flareon", "porygon", "omanyte", "omastar", "kabuto", "kabutops", "aerodactyl",
    "snorlax", "articuno", "zapdos", "moltres", "dratini", "dragonair", "dragonite",
    "mewtwo", "mew","chikorita","bayleef","meganium","cyndaquil","quilava","typhlosion",
"totodile","croconaw","feraligatr","sentret","furret","hoothoot",
"noctowl","ledyba","ledian","spinarak","ariados","crobat",
"chinchou","lanturn","pichu","cleffa","igglybuff","togepi",
"togetic","natu","xatu","mareep","flaaffy","ampharos",
"bellossom","marill","azumarill","sudowoodo","politoed","hoppip",
"skiploom","jumpluff","aipom","sunkern","sunflora","yanma",
"wooper","quagsire","espeon","umbreon","murkrow","slowking",
"misdreavus","unown","wobbuffet","girafarig","pineco","forretress",
"dunsparce","gligar","steelix","snubbull","granbull","qwilfish",
"scizor","shuckle","heracross","sneasel","teddiursa","ursaring",
"slugma","magcargo","swinub","piloswine","corsola","remoraid",
"octillery","delibird","mantine","skarmory","houndour","houndoom",
"kingdra","phanpy","donphan","porygon","stantler","smeargle",
"tyrogue","hitmontop","smoochum","elekid","magby","miltank",
"blissey","raikou","entei","suicune","larvitar","pupitar",
"tyranitar","lugia","hooh","celebi"
]

with open('pokemon.txt', 'w') as f:
    for name in pokemon_list:
        f.write(name + '\n')

In [ ]:
words = open('pokemon.txt', 'r').read().splitlines()

words = [w.strip().lower() for w in words if w.strip()]


In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size=len(chars)+1

xs, ys = [], []
for w in words:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1, ix2, ix3 = stoi[ch1], stoi[ch2], stoi[ch3]
        xs.append([ix1, ix2])
        ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement() // 2
print(f'Number of examples: {num}')
vocab_size
itos

Number of examples: 2067


{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [ ]:
g = torch.Generator().manual_seed(2147483637)
W = torch.randn((54, 27), generator=g, requires_grad=True)


for k in range(2000):

    xenc = F.one_hot(xs, num_classes=27).float().view(-1, 54)
    logits = xenc @ W

    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)

    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()


    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

    if k % 200 == 0:
        print(f"step {k}: loss {loss.item():.4f}")

step 0: loss 4.2292
step 200: loss 2.3018
step 400: loss 2.2937
step 600: loss 2.2921
step 800: loss 2.2916
step 1000: loss 2.2914
step 1200: loss 2.2913
step 1400: loss 2.2912
step 1600: loss 2.2912
step 1800: loss 2.2912


In [ ]:
temp = 0.7

for _ in range(15):
    out = []
    context = [0, 0]

    while True:
        xenc = F.one_hot(torch.tensor(context), num_classes=vocab_size).float().view(1, -1)
        logits = xenc @ W
        logits = logits / temp
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, 1).item()
        context = context[1:] + [ix]
        if ix == 0:
            break
        out.append(itos[ix])

    print("".join(out))

sout
mar
jin
arig
heon
avuntochum
mrggorasnybullee
ume
akacrabratreor
ariwhoning
sil
seen
suedichyh
pidoleandshon
azabrytanod
